# Block 6 Exercise 2: finding the best parameters for predicting the fare of taxi rides
We return to our Random Forest Regression and want to automatically optimize all free parameters ...

In [3]:
!pip install folium

     |████████████████████████████████| 94 kB 3.8 MB/s eta 0:00:011


In [4]:
import pandas as pd
import numpy as np
import folium


In [5]:
# we load the data we have saved after wrangling and pre-processing in block I
X=pd.read_csv('../../DATA/train_cleaned.csv')
drop_columns=['Unnamed: 0','Unnamed: 0.1','Unnamed: 0.1.1','key','pickup_datetime','pickup_date','pickup_latitude_round3','pickup_longitude_round3','dropoff_latitude_round3','dropoff_longitude_round3']
X=X.drop(drop_columns,axis=1)
X=pd.get_dummies(X)# one hot coding
#generate labels
y=X['fare_amount']
X=X.drop(['fare_amount'],axis=1)

### Scikit Optimize
Scikit Optimize (https://scikit-optimize.github.io/stable/index.html) is a AutoML toolbox wrapped around Scikit-Learn. It allows us to use state-of-the-art automatic hyper-parameter optimization on top of our learning algorithms.   



In [6]:
# install 
!pip install scikit-optimize

     |████████████████████████████████| 101 kB 5.8 MB/s eta 0:00:01


### E 2.1 Bayesian Optimization of a Random Forest Regression Model
use Bayesian Optimization with Cross-Validation (https://scikit-optimize.github.io/stable/modules/generated/skopt.BayesSearchCV.html#skopt.BayesSearchCV) to find the best regression model. Compare
* linear regression (https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html#sklearn.linear_model.LinearRegression) 
* Random Forest regression (https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestRegressor.html#sklearn.ensemble.RandomForestRegressor)
* and SVM regression (https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVR.html#sklearn.svm.SVR)

NOTES: this can become quite compute intensive! Hence,
* use a smaller subset of the training data to run the experiments 
* think about the range of your parameters (e.g. larger number of trees in RF or high C-values in SMV will make models expensive)
* optimize only the following parameters per model type:
    * linear: no parameters to optimize
    * RF: #trees and depth
    * SVM: C and gamma (use RBF kernel)
* parallelize -> n_jobs
* use CoLab to rum the job for up to 12h 


In [31]:
from skopt import BayesSearchCV as BSCV
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression as LR
from sklearn.ensemble import RandomForestRegressor as RFR
from sklearn.svm import SVR 
from sklearn.metrics import mean_squared_error as mse

In [14]:
np.shape(X)

(400000, 31)

In [19]:
y_less = y[:10000]
X_less=X[:10000]
print(np.shape(X_less), np.shape(y_less))

(10000, 31) (10000,)


In [23]:
X_l_train,X_l_test,y_l_train,y_l_test = train_test_split(X_less,y_less, test_size=0.1, random_state = 1)

In [28]:
print(type(y_l_train), type(y_l_test))

<class 'pandas.core.series.Series'> <class 'pandas.core.series.Series'>


***Linear regression.***
- There are no parameters to be optimzied. So no BayesSearchCV is used

In [24]:
LRm = LR().fit(X_l_train,y_l_train)

In [29]:
res_LR_y_l_test = LRm.predict(X_l_test)

In [33]:
mse_LR_test = mse(y_l_test,res_LR_y_l_test)
mse_LR_test

25.88405237662758

***Random Forest Tree***